# Insurance Policy Chatbot

The main objective of this notebook is to guide the reader through the process of understanding how the chatbot works and was build.

The main functionality is to responde to the user's questions about insurance policies based on a PDF database (and WEB search).

The beggining of the process start with the download of the PDFs from AWS S3 into the local storage (_folder:_ `backend/database_s3/queplan_insurance`), then we can start generating our embedding vectors for each of the documents.

In [1]:
# Download the PDF files
from backend.download import download_data

download_data()

ModuleNotFoundError: No module named 'utils'

That was an intended error.

The architecture of the project intend to be converted into a microservice architecture, and so, it is necesary to use the `os` and `sys` libraries to indicate the microservice root path for executing the files.

In [2]:
import sys, os

service = "backend"
sys.path.insert(0, os.path.join(os.path.dirname('.'), service))

from backend.download import download_data

download_data()

Download queplan_insurance/POL120190177.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL120190177.pdf
Download queplan_insurance/POL320130223.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320130223.pdf
Download queplan_insurance/POL320150503.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320150503.pdf
Download queplan_insurance/POL320180100.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320180100.pdf
Download queplan_insurance/POL320190074.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320190074.pdf
Download queplan_insurance/POL320200071.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320200071.pdf
Download queplan_insurance/POL320200214.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\quepla

# 1. EDA and Summarization

One of the main goals of the backend is to provide resources that allow the system to transform inputs into a wealthy context to feed an LLM.

For that we implemented two processes:
- **EDA:** using regular expression with the `re` library to extract the main patterns of the documents. The EDA is a exploratory analysis focused on identifying relevant keywords and formatting patterns.

- **Summarization:** using the OpenAI LLM `gpt-3-turbo` (16k) and `langchain` document chains to obtain a bullet point summary of each page of the document. The summarization is a process that aims to reduce bias by obtaining a semantic aproximation of each of the pages.

#### Summary:

So, antes de que la consulta del usuario vaya a la base de datos vectorizada donde estan todos los documentos, lo que hacemos es evaluar dicha consulta con una serie de features generadas por nosotros.
Basado en nuestros vectores de similitud y los vectores de semantica generados por nuestras features podemos traer los documentos o las paginas que mas se asemejen a la consulta del usuario. Y como la metadata de nuestras features es compartida en parte con la metadata de los documentos, utilizamos dicha informacion para obtener los segmentos más relevantes.

#### Benefits:

Nuestro trabajo es alimentar a un modelo LLM con el input del usuario y la mejor calidad posible de contexto. Este contexto no es infinito, esta limitado, por lo tanto si conseguimos separar el contenido mas relevante relacionado a la consulta del usuario, el contexto podrá ser más extenso, y el modelo más preciso.




In [7]:
# Example of an output of Summarization process

# document index = 0
# page index = 0
# prompt_template = """Extraer los bullet-points de los articulos\
#  contenidos en el TEXTO. La respuesta debe ser de 1000 caracteres como maximo.
# TEXTO:
# "{text}"
# RESUMEN:"""

import pickle

response_path = f"{service}/.summarization_backup/"

with open(response_path + "response_1709557379.pkl", "rb") as f:
    response = pickle.load(f)
    
print(response["output_text"])

- El contrato de seguro se regirá por las disposiciones legales y las estipulaciones más beneficiosas para el asegurado o beneficiario.
- La cobertura de la póliza incluye el reembolso de los gastos médicos razonables y acostumbrados incurridos por el asegurado en caso de accidente.
- Los gastos deben ser realizados en los prestadores de salud determinados por la aseguradora y durante el período de duración del reembolso.
- Se establecen requisitos para el reembolso de los gastos, como que el accidente haya ocurrido durante la vigencia de la póliza, que se haya incurrido en los gastos dentro del período de duración de reembolso, que el monto de gastos supere el deducible establecido, entre otros.
- Los reembolsos están sujetos a límites y topes establecidos en las condiciones particulares de la póliza, así como a porcentajes de bonificación o reembolso entregados por el sistema de salud previsional al que esté afiliado el asegurado.
- La aseguradora reembolsará los gastos médicos como 

----

La parte del EDA es bastante más compleja, primero se hace una exploración completa del documento.
El primer punto es obtener todos los artículos que figuran dentro del documento. Si conseguimos hacer esto ya tenemos un vector que represente el contenido de todo el documento.

Acá hay un ejemplo de como crear una feature que represente a un documento completo:


In [8]:
import sys, os

service = "backend"
sys.path.insert(0, os.path.join(os.path.dirname('.'), service))


from backend.eda_tools.pdf_file_tools import extract_patterns
from backend.utils.config import DOWNLOAD_PATH

file_n = 0 
pdf_files = os.listdir(DOWNLOAD_PATH)
selected_file = os.path.join(DOWNLOAD_PATH, pdf_files[file_n])
pattern = "ART.CULO\s\d+.+:"

print(selected_file.split("\\")[-1])
extract_patterns(selected_file, pattern)

POL120190177.pdf


['ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO',
 'ARTÍCULO 2º: COBERTURA Y MATERIA ASEGURADA',
 'ARTÍCULO 3°: LIMITACIONES DE LAS COBERTURAS',
 'ARTÍCULO 4º: DEFINICIONES',
 'ARTÍCULO 5º: EXCLUSIONES',
 'ARTÍCULO 6°: OBLIGACIONES DEL ASEGURADO',
 'ARTÍCULO 7°: DECLARACIONES DEL CONTRATANTE Y DEL ASEGURADO',
 'ARTÍCULO 8º: PRIMAS Y EFECTO DEL NO PAGO DE LA PRIMA',
 'ARTÍCULO 9º: DENUNCIA DE SINIESTRO ',
 'ARTÍCULO 10°: CALCULO DE LOS GASTOS REEMBOLSABLES',
 'ARTÍCULO 11°: LIQUIDACIÓN DE LOS GASTOS, FORMA DE PAGO Y MONTO MÁXIMO DE GASTOS',
 'ARTÍCULO 12°: APLICACIÓN DEL DEDUCIBLE',
 'ARTÍCULO 13°: VIGENCIA Y DURACIÓN DEL CONTRATO DE SEGURO',
 'ARTÍCULO 14°: INCORPORACIÓN DE ASEGURADOS E INICIO DE LA VIGENCIA DE LA COBERTURA',
 'ARTÍCULO 15°: TERMINACIÓN DEL CONTRATO',
 'ARTÍCULO 16º: AJUSTE DE LA PRIMAEn el evento que, con motivo de una renovación de póliza, las primas sufran una modificación en su monto',
 'ARTÍCULO 17°: MONEDA O UNIDAD DEL CONTRATO',
 'ARTICULO 18°: REHABILITACIÓN DE LA

In [9]:
# Técnicamente debería ser así:
# porque no necesitamos una lista con elementos
# necesitamos un string
# dicho string es la feature en lenguaje natural de todo el documento
# Pero es menos legible por acá 

" ".join(extract_patterns(selected_file, pattern))

'ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO ARTÍCULO 2º: COBERTURA Y MATERIA ASEGURADA ARTÍCULO 3°: LIMITACIONES DE LAS COBERTURAS ARTÍCULO 4º: DEFINICIONES ARTÍCULO 5º: EXCLUSIONES ARTÍCULO 6°: OBLIGACIONES DEL ASEGURADO ARTÍCULO 7°: DECLARACIONES DEL CONTRATANTE Y DEL ASEGURADO ARTÍCULO 8º: PRIMAS Y EFECTO DEL NO PAGO DE LA PRIMA ARTÍCULO 9º: DENUNCIA DE SINIESTRO  ARTÍCULO 10°: CALCULO DE LOS GASTOS REEMBOLSABLES ARTÍCULO 11°: LIQUIDACIÓN DE LOS GASTOS, FORMA DE PAGO Y MONTO MÁXIMO DE GASTOS ARTÍCULO 12°: APLICACIÓN DEL DEDUCIBLE ARTÍCULO 13°: VIGENCIA Y DURACIÓN DEL CONTRATO DE SEGURO ARTÍCULO 14°: INCORPORACIÓN DE ASEGURADOS E INICIO DE LA VIGENCIA DE LA COBERTURA ARTÍCULO 15°: TERMINACIÓN DEL CONTRATO ARTÍCULO 16º: AJUSTE DE LA PRIMAEn el evento que, con motivo de una renovación de póliza, las primas sufran una modificación en su monto ARTÍCULO 17°: MONEDA O UNIDAD DEL CONTRATO ARTICULO 18°: REHABILITACIÓN DE LA PÓLIZA ARTÍCULO 19º: CONTRIBUCIONES E IMPUESTOS  ARTÍCULO 20º: COMUN

-----

Luego tenemos una versión similar de dicha función, pero en vez de tratar todo el documento lo hace pagina por pagina. Además extraer las primeras 4 lineas posteriores al patron identificado.

De ese modo se apunta a obtener una feature de cada página.

In [11]:
import sys, os

service = "backend"
sys.path.insert(0, os.path.join(os.path.dirname('.'), service))


from backend.eda_tools.pdf_file_tools import extract_patterns_per_page
from backend.utils.config import DOWNLOAD_PATH

file_n = 0
pdf_files = os.listdir(DOWNLOAD_PATH)
selected_file = os.path.join(DOWNLOAD_PATH, pdf_files[file_n])
pattern = "ART.CULO\s\d+.+:"

result = extract_patterns_per_page(selected_file, pattern, extend=True)
result

[['ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO',
  'Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las',
  'normas legales de carácter imperativo establecidas en el título VIII, del Libro II, del Código de Comercio. Sin',
  'embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el',
  'asegurado o el beneficiario.',
  'ARTÍCULO 2º: COBERTURA Y MATERIA ASEGURADA',
  'La Compañía Aseguradora reembolsará al asegurado o pagará directamente al prestador de salud los',
  'Gastos Médicos Razonables y Acostumbrados y Efectivamente Incurridos, una vez se haya otorgado y',
  'pagado la cobertura del sistema de salud previsional, seguros complementarios u otros beneficios',
  'contratados por el asegurado. Lo anterior, cuando al asegurado le ocurra un accidente durante la vigencia de'],
 [],
 ['ARTÍCULO 3°: LIMITACIONES DE LAS COBERTURAS',
  'Sin perjuicio de los términos, porcentajes, límites y topes 

Por ejemplo podemos ver que la primera página tiene 2 artículos, la segunda no tiene ninguno y que la tercera tiene 2, y que la cuarta y la quinta no tienen ninguno.

In [12]:
# En realidad las features deberían ser un string

[" ".join(element) for element in result]

['ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las normas legales de carácter imperativo establecidas en el título VIII, del Libro II, del Código de Comercio. Sin embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el asegurado o el beneficiario. ARTÍCULO 2º: COBERTURA Y MATERIA ASEGURADA La Compañía Aseguradora reembolsará al asegurado o pagará directamente al prestador de salud los Gastos Médicos Razonables y Acostumbrados y Efectivamente Incurridos, una vez se haya otorgado y pagado la cobertura del sistema de salud previsional, seguros complementarios u otros beneficios contratados por el asegurado. Lo anterior, cuando al asegurado le ocurra un accidente durante la vigencia de',
 '',
 'ARTÍCULO 3°: LIMITACIONES DE LAS COBERTURAS Sin perjuicio de los términos, porcentajes, límites y topes de reembolso que se establezcan en el Cuadro de Cober

In [ ]:
# Notas: please revisar

"""
Notas y conclusiones:

- [0]: POL120190177.pdf y [2]: POL320150503.pdf y [3]: POL320180100.pdf parecen iguales
- [1]: POL320130223.pdf y [5]: POL320200071.pdf error de patron
- [4]: POL320190074.pdf error en la numeracion de los articulos
- [7]: POL320200214.pdf falta el articulo 9

""";